# The Bayesian Network approach is discarded

In [ ]:
import pandas as pd
import numpy as np
from pgmpy.factors.discrete import TabularCPD

In [ ]:
class BayesianCPTBuilder:
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)
        self.df = self.df.rename(columns={
            'gender': 'Gender',
            'feature': 'Feature',
            'median_income': 'IncomeLevel',
            'PROPORTION': 'Proportion'
        })
        self.genders = sorted(self.df['Gender'].unique())
        self.feature_groups = sorted(self.df['Feature'].unique())
        self.income_levels = sorted(self.df['IncomeLevel'].unique())
        self.income_levels = ['<10k', '10k-20k', '20k-30k', '30k-50k', '50k-70k', '70k-90k', '90k-110k', '110k-130k', '130-150k', '>150k']

    def build_feature_given_gender_cpd(self, feature_name):
        cpt_matrix = []
        for feature in self.feature_groups:
            row = []
            for gender in self.genders:
                prob = self.df[
                    (self.df['Gender'] == gender) & (self.df['Feature'] == feature)
                ]['Proportion'].values
                row.append(prob[0] if len(prob) > 0 else 0.0)
            cpt_matrix.append(row)

        return TabularCPD(
            variable=feature_name,
            variable_card=len(self.feature_groups),
            values=cpt_matrix,
            evidence=['Gender'],
            evidence_card=[len(self.genders)],
            state_names={
                feature_name: self.feature_groups,
                'Gender': self.genders
            }
        )
        
    def build_income_given_feature_gender_cpd(self, feature_name):
        cpt_matrix = []
        for income in self.income_levels:
            row = []
            for gender in self.genders:
                for feature in self.feature_groups:
                    prob = self.df[
                        (self.df['Gender'] == gender) &
                        (self.df['Feature'] == feature) &
                        (self.df['IncomeLevel'] == income)
                    ]['Proportion'].sum()
                    row.append(prob)
            cpt_matrix.append(row)

        import numpy as np
        cpt_array = np.array(cpt_matrix)
        col_sums = cpt_array.sum(axis=0)
        normalized_matrix = (cpt_array / col_sums).tolist()

        return TabularCPD(
            variable='IncomeLevel',
            variable_card=len(self.income_levels),
            values=normalized_matrix,
            evidence=[feature_name, 'Gender'],
            evidence_card=[len(self.feature_groups), len(self.genders)],
            state_names={
                'IncomeLevel': self.income_levels,
                feature_name: self.feature_groups,
                'Gender': self.genders
            }
        )

In [ ]:
def fix_cpd_normalization(cpd: TabularCPD) -> TabularCPD:
    arr = np.array(cpd.values)
    # arr: shape = (variable_card, n_cols)
    for col in range(arr.shape[1]):
        s = arr[:, col].sum()
        if np.isclose(s, 0):
            arr[:, col] = 1.0 / arr.shape[0]
        else:
            arr[:, col] = arr[:, col] / s
    return TabularCPD(
        variable=cpd.variable,
        variable_card=cpd.variable_card,
        values=arr.tolist(),
        evidence=cpd.variables[1:],
        evidence_card=cpd.cardinality[1:],
        state_names=cpd.state_names
    )


In [ ]:
# BN structure
from pgmpy.models import DiscreteBayesianNetwork

builder_age = BayesianCPTBuilder("../../data/randomized marginal tables/age_gender_randomized.csv")
gender_proportions = builder_age.df.groupby('Gender')['Proportion'].sum()
cpd_age_gender = builder_age.build_feature_given_gender_cpd('AgeGroup')
cpd_age_gender = fix_cpd_normalization(cpd_age_gender)
cpd_age_gender.normalize()

builder_industry = BayesianCPTBuilder("../../data/randomized marginal tables/industry_gender_randomized.csv")
cpd_industry_gender = builder_industry.build_feature_given_gender_cpd('Industry')
cpd_industry_gender = fix_cpd_normalization(cpd_industry_gender)
cpd_industry_gender.normalize()

builder_corp_size = BayesianCPTBuilder("../../data/randomized marginal tables/corpSize_gender_randomized.csv")
cpd_corp_size_gender = builder_corp_size.build_feature_given_gender_cpd('CorpSize')
cpd_corp_size_gender = fix_cpd_normalization(cpd_corp_size_gender)
cpd_corp_size_gender.normalize()

builder_employmentType = BayesianCPTBuilder("../../data/randomized marginal tables/employmentType_gender_randomized.csv")
cpd_employment_type_gender = builder_employmentType.build_feature_given_gender_cpd('EmploymentType')
cpd_employment_type_gender = fix_cpd_normalization(cpd_employment_type_gender)
cpd_employment_type_gender.normalize()

builder_legal_org = BayesianCPTBuilder("../../data/randomized marginal tables/legal_org_gender_randomized.csv")
cpd_legal_org_gender = builder_legal_org.build_feature_given_gender_cpd('LegalOrganization')
cpd_legal_org_gender = fix_cpd_normalization(cpd_legal_org_gender)
cpd_legal_org_gender.normalize()

builder_sector = BayesianCPTBuilder("../../data/randomized marginal tables/sector_gender_randomized.csv")
cpd_sector_gender = builder_sector.build_feature_given_gender_cpd('Sector')
cpd_sector_gender = fix_cpd_normalization(cpd_sector_gender)
cpd_sector_gender.normalize()

cpd_gender = TabularCPD(
    variable='Gender',
    variable_card=2,
    values=[[gender_proportions['FEMALES']], [gender_proportions['MALES']]],
    state_names={'Gender': ['FEMALES', 'MALES']}
)
cpd_gender.normalize()

model = DiscreteBayesianNetwork()
model.add_edges_from([
    ('Gender', 'AgeGroup'),
    ('Gender', 'Industry'),
    ('Gender', 'CorpSize'),
    ('Gender', 'EmploymentType'),
    ('Gender', 'LegalOrganization'),
    ('Gender', 'Sector'),
    ('AgeGroup', 'IncomeLevel'),
    ('Industry', 'IncomeLevel'),
    ('CorpSize', 'IncomeLevel'),
    ('EmploymentType', 'IncomeLevel'),
    ('LegalOrganization', 'IncomeLevel'),
    ('Sector', 'IncomeLevel')
])

model.add_cpds(cpd_gender)
model.add_cpds(cpd_age_gender, cpd_industry_gender)
model.add_cpds(cpd_corp_size_gender, cpd_employment_type_gender, cpd_legal_org_gender, cpd_sector_gender)


In [ ]:
import pandas as pd
import itertools

def get_prob_table(builder, feature_col):
    prob_df = builder.df[['Feature', 'Gender', 'IncomeLevel', 'Proportion']].copy()
    prob_df = prob_df.rename(columns={
        'Feature': feature_col,
        'Proportion': f'Prob_{feature_col}'
    })
    return prob_df

prob_age = get_prob_table(builder_age, 'AgeGroup')
prob_industry = get_prob_table(builder_industry, 'Industry')
prob_corp_size = get_prob_table(builder_corp_size, 'CorpSize')
prob_employment_type = get_prob_table(builder_employmentType, 'EmploymentType')
prob_legal_org = get_prob_table(builder_legal_org, 'LegalOrganization')
prob_sector = get_prob_table(builder_sector, 'Sector')

In [ ]:
age_groups = sorted(builder_age.df['Feature'].unique())
industry_groups = sorted(builder_industry.df['Feature'].unique())
corp_size_groups = sorted(builder_corp_size.df['Feature'].unique())
employment_types = sorted(builder_employmentType.df['Feature'].unique())
legal_orgs = sorted(builder_legal_org.df['Feature'].unique())
sectors = sorted(builder_sector.df['Feature'].unique())
genders = sorted(builder_age.df['Gender'].unique())
income_levels = sorted(builder_age.df['IncomeLevel'].unique())

combinations = list(itertools.product(
    age_groups, industry_groups, corp_size_groups,
    employment_types, legal_orgs, sectors, genders, income_levels
))
df_joint7 = pd.DataFrame(combinations, columns=[
    'AgeGroup', 'Industry', 'CorpSize', 'EmploymentType',
    'LegalOrganization', 'Sector', 'Gender', 'IncomeLevel'
])

df = df_joint7
df = df.merge(prob_age, on=['AgeGroup', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_industry, on=['Industry', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_corp_size, on=['CorpSize', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_employment_type, on=['EmploymentType', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_legal_org, on=['LegalOrganization', 'Gender', 'IncomeLevel'], how='left')
df = df.merge(prob_sector, on=['Sector', 'Gender', 'IncomeLevel'], how='left')

In [ ]:
df['Proportion'] = (
    df['Prob_AgeGroup'].fillna(0) *
    df['Prob_Industry'].fillna(0) *
    df['Prob_CorpSize'].fillna(0) *
    df['Prob_EmploymentType'].fillna(0) *
    df['Prob_LegalOrganization'].fillna(0) *
    df['Prob_Sector'].fillna(0)
)

group_cols = ['AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector', 'Gender']
df['Proportion'] = df.groupby(group_cols)['Proportion'].transform(lambda x: x / x.sum())

df_joint7 = df

In [ ]:
def build_income_given_all_features_cpd_fast(
    df_joint7, income_levels, age_groups, industry_groups, corp_size_groups,
    employment_types, legal_orgs, sectors
):
    all_index = pd.MultiIndex.from_product(
        [income_levels, age_groups, industry_groups, corp_size_groups, employment_types, legal_orgs, sectors],
        names=['IncomeLevel', 'AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector']
    )
    pivot = df_joint7.pivot_table(
        index=['IncomeLevel', 'AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector'],
        values='Proportion',
        fill_value=0
    ).reindex(all_index, fill_value=0)

    shape = (
        len(income_levels),
        len(age_groups),
        len(industry_groups),
        len(corp_size_groups),
        len(employment_types),
        len(legal_orgs),
        len(sectors)
    )
    arr = pivot.values.reshape(shape)
    cpt_matrix = arr.reshape(len(income_levels), -1)

    import numpy as np
    col_sums = cpt_matrix.sum(axis=0)
    normalized_matrix = []
    for col in range(cpt_matrix.shape[1]):
        col_sum = col_sums[col]
        if np.isclose(col_sum, 0):
            normalized_matrix.append([1.0 / len(income_levels)] * len(income_levels))
        else:
            normalized_matrix.append((cpt_matrix[:, col] / col_sum).tolist())
    normalized_matrix = np.array(normalized_matrix).T.tolist()

    return TabularCPD(
        variable='IncomeLevel',
        variable_card=len(income_levels),
        values=normalized_matrix,
        evidence=['AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector'],
        evidence_card=[
            len(age_groups), len(industry_groups), len(corp_size_groups),
            len(employment_types), len(legal_orgs), len(sectors)
        ],
        state_names={
            'IncomeLevel': income_levels,
            'AgeGroup': age_groups,
            'Industry': industry_groups,
            'CorpSize': corp_size_groups,
            'EmploymentType': employment_types,
            'LegalOrganization': legal_orgs,
            'Sector': sectors
        }
    )
    
cpd_income_all = build_income_given_all_features_cpd_fast(
    df_joint7, income_levels, age_groups, industry_groups, corp_size_groups,
    employment_types, legal_orgs, sectors
)

model.add_cpds(cpd_income_all)
assert model.check_model()

In [23]:
cpd_income_all.to_csv("BN_income_all_cpd.csv")

In [ ]:
from pgmpy.sampling import BayesianModelSampling

sampling_inference = BayesianModelSampling(model)
samples2 = sampling_inference.forward_sample(size=100000)
print(samples2.head())
samples2.to_csv("../../data/synthetic/BN1.0_samples_100000.csv", index=False)


  0%|          | 0/8 [00:00<?, ?it/s]

    Gender        AgeGroup                           Industry  \
0    MALES  18 to 20 years  Agriculture, forestry and fishing   
1  FEMALES  15 to 17 years    Accommodation and food services   
2    MALES  18 to 20 years  Agriculture, forestry and fishing   
3    MALES  18 to 20 years    Accommodation and food services   
4  FEMALES  75 to 79 years       Arts and recreation services   

                 CorpSize                          EmploymentType  \
0  Fewer than 5 employees  Unincorporated Private sector entities   
1  Fewer than 5 employees  Unincorporated Private sector entities   
2        20–199 employees                  Public sector entities   
3        20–199 employees                  Public sector entities   
4          5–19 employees  Unincorporated Private sector entities   

                            LegalOrganization          Sector IncomeLevel  
0                                  Households  Other services     90-110k  
1                                  Househo

In [16]:
import pandas as np
test_data = np.read_csv("../../data/synthetic/BN1.0_samples_100000.csv")

In [24]:
test_data['IncomeLevel'].value_counts(normalize=True)

110-130k    0.10153
130-150k    0.10085
50-70k      0.10080
20-30k      0.10048
>150k       0.10029
30-50k      0.09959
10-20k      0.09958
70-90k      0.09911
90-110k     0.09908
<10k        0.09869
Name: IncomeLevel, dtype: float64

In [ ]:
# Solution 1: Redesign the Bayesian network structure
# Use a more reasonable structure: Gender -> Features -> Income
# Instead of having all features directly point to Income

import pandas as pd
import numpy as np
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD

class ImprovedBayesianCPTBuilder:
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)
        self.df = self.df.rename(columns={
            'gender': 'Gender',
            'feature': 'Feature',
            'median_income': 'IncomeLevel',
            'PROPORTION': 'Proportion'
        })
        self.genders = sorted(self.df['Gender'].unique())
        self.feature_groups = sorted(self.df['Feature'].unique())
        self.income_levels = sorted(self.df['IncomeLevel'].unique())
        
    def build_feature_given_gender_cpd(self, feature_name):
        cpt_matrix = []
        for feature in self.feature_groups:
            row = []
            for gender in self.genders:
                gender_feature_prob = self.df[
                    (self.df['Gender'] == gender) & (self.df['Feature'] == feature)
                ]['Proportion'].sum()
                row.append(gender_feature_prob)
            cpt_matrix.append(row)
        
        cpt_matrix = np.array(cpt_matrix)
        col_sums = cpt_matrix.sum(axis=0)
        for col in range(cpt_matrix.shape[1]):
            if col_sums[col] > 0:
                cpt_matrix[:, col] = cpt_matrix[:, col] / col_sums[col]
            else:
                cpt_matrix[:, col] = 1.0 / len(self.feature_groups)
        
        return TabularCPD(
            variable=feature_name,
            variable_card=len(self.feature_groups),
            values=cpt_matrix.tolist(),
            evidence=['Gender'],
            evidence_card=[len(self.genders)],
            state_names={
                feature_name: self.feature_groups,
                'Gender': self.genders
            }
        )
    
    def build_income_given_feature_gender_cpd(self, feature_name):
        cpt_matrix = []
        for income in self.income_levels:
            row = []
            for gender in self.genders:
                for feature in self.feature_groups:
                    prob = self.df[
                        (self.df['Gender'] == gender) &
                        (self.df['Feature'] == feature) &
                        (self.df['IncomeLevel'] == income)
                    ]['Proportion'].values
                    
                    if len(prob) > 0:
                        row.append(prob[0])
                    else:
                        row.append(0.0)
            cpt_matrix.append(row)
        
        cpt_matrix = np.array(cpt_matrix)
        col_sums = cpt_matrix.sum(axis=0)
        for col in range(cpt_matrix.shape[1]):
            if col_sums[col] > 0:
                cpt_matrix[:, col] = cpt_matrix[:, col] / col_sums[col]
            else:
                cpt_matrix[:, col] = 1.0 / len(self.income_levels)
        
        return TabularCPD(
            variable='IncomeLevel',
            variable_card=len(self.income_levels),
            values=cpt_matrix.tolist(),
            evidence=[feature_name, 'Gender'],
            evidence_card=[len(self.feature_groups), len(self.genders)],
            state_names={
                'IncomeLevel': self.income_levels,
                feature_name: self.feature_groups,
                'Gender': self.genders
            }
        )


In [ ]:
# Solution 2: Use smarter joint probability estimation
# Build joint distribution based on actually observed patterns

class SmartJointDistributionBuilder:
    def __init__(self, marginal_files):
        self.marginal_data = {}
        self.load_marginal_data(marginal_files)
        
    def load_marginal_data(self, marginal_files):
        for feature_name, filepath in marginal_files.items():
            df = pd.read_csv(filepath)
            df = df.rename(columns={
                'gender': 'Gender',
                'feature': 'Feature',
                'median_income': 'IncomeLevel',
                'PROPORTION': 'Proportion'
            })
            self.marginal_data[feature_name] = df
    
    def build_joint_distribution(self):
        all_features = {}
        for feature_name, df in self.marginal_data.items():
            all_features[feature_name] = sorted(df['Feature'].unique())
        
        genders = sorted(self.marginal_data[list(self.marginal_data.keys())[0]]['Gender'].unique())
        income_levels = sorted(self.marginal_data[list(self.marginal_data.keys())[0]]['IncomeLevel'].unique())
        
        import itertools
        feature_combinations = list(itertools.product(*all_features.values()))
        
        joint_probs = []
        
        for gender in genders:
            for feature_combo in feature_combinations:
                for income in income_levels:
                    prob = self.calculate_joint_probability(
                        gender, dict(zip(all_features.keys(), feature_combo)), income
                    )
                    joint_probs.append({
                        'Gender': gender,
                        'IncomeLevel': income,
                        **dict(zip(all_features.keys(), feature_combo)),
                        'JointProbability': prob
                    })
        
        return pd.DataFrame(joint_probs)
    
    def calculate_joint_probability(self, gender, feature_values, income):
        p_gender = 0.5
        p_features_given_gender = 1.0
        for feature_name, feature_value in feature_values.items():
            df = self.marginal_data[feature_name]
            feature_prob = df[
                (df['Gender'] == gender) & (df['Feature'] == feature_value)
            ]['Proportion'].sum()
            p_features_given_gender *= feature_prob
        
        primary_feature = 'AgeGroup' if 'AgeGroup' in feature_values else list(feature_values.keys())[0]
        primary_value = feature_values[primary_feature]
        
        df = self.marginal_data[primary_feature]
        income_prob = df[
            (df['Gender'] == gender) & 
            (df['Feature'] == primary_value) & 
            (df['IncomeLevel'] == income)
        ]['Proportion'].values
        
        if len(income_prob) > 0:
            p_income_given_gender_primary = income_prob[0]
        else:
            gender_income_probs = df[
                (df['Gender'] == gender) & (df['IncomeLevel'] == income)
            ]['Proportion'].sum()
            total_gender_prob = df[df['Gender'] == gender]['Proportion'].sum()
            p_income_given_gender_primary = gender_income_probs / total_gender_prob if total_gender_prob > 0 else 1.0 / len(df['IncomeLevel'].unique())
        
        joint_prob = p_gender * p_features_given_gender * p_income_given_gender_primary
        
        return joint_prob


In [ ]:
# Test Solution 2: Use Smart Joint Distribution Builder

# Define marginal file paths
marginal_files = {
    'AgeGroup': "../../data/randomized marginal tables/age_gender_randomized.csv",
    'Industry': "../../data/randomized marginal tables/industry_gender_randomized.csv",
    'CorpSize': "../../data/randomized marginal tables/corpSize_gender_randomized.csv",
    'EmploymentType': "../../data/randomized marginal tables/employmentType_gender_randomized.csv",
    'LegalOrganization': "../../data/randomized marginal tables/legal_org_gender_randomized.csv",
    'Sector': "../../data/randomized marginal tables/sector_gender_randomized.csv"
}

smart_builder = SmartJointDistributionBuilder(marginal_files)

print("Joint distribution...")
joint_df = smart_builder.build_joint_distribution()
print(f"Joint distribution built with {len(joint_df)} combinations")

# Check the quality of the joint distribution
print("\nJoint distribution statistics:")
print(f"Total probability: {joint_df['JointProbability'].sum():.6f}")
print(f"Non-zero probability combinations: {(joint_df['JointProbability'] > 0).sum()}")
print(f"Maximum probability: {joint_df['JointProbability'].max():.6f}")
print(f"Minimum non-zero probability: {joint_df[joint_df['JointProbability'] > 0]['JointProbability'].min():.6f}")

# Check income distribution
income_dist = joint_df.groupby('IncomeLevel')['JointProbability'].sum().sort_values(ascending=False)
print("\nIncome level distribution:")
print(income_dist)


In [ ]:
# Solution 3: Build an improved Bayesian network
# Use a more reasonable network structure to avoid all features directly pointing to income

def build_improved_bayesian_network():
    builders = {}
    cpds = {}
    
    builders['AgeGroup'] = ImprovedBayesianCPTBuilder("../../data/randomized marginal tables/age_gender_randomized.csv")
    cpds['AgeGroup'] = builders['AgeGroup'].build_feature_given_gender_cpd('AgeGroup')
    
    builders['Industry'] = ImprovedBayesianCPTBuilder("../../data/randomized marginal tables/industry_gender_randomized.csv")
    cpds['Industry'] = builders['Industry'].build_feature_given_gender_cpd('Industry')
    
    builders['CorpSize'] = ImprovedBayesianCPTBuilder("../../data/randomized marginal tables/corpSize_gender_randomized.csv")
    cpds['CorpSize'] = builders['CorpSize'].build_feature_given_gender_cpd('CorpSize')
    
    builders['EmploymentType'] = ImprovedBayesianCPTBuilder("../../data/randomized marginal tables/employmentType_gender_randomized.csv")
    cpds['EmploymentType'] = builders['EmploymentType'].build_feature_given_gender_cpd('EmploymentType')
    
    builders['LegalOrganization'] = ImprovedBayesianCPTBuilder("../../data/randomized marginal tables/legal_org_gender_randomized.csv")
    cpds['LegalOrganization'] = builders['LegalOrganization'].build_feature_given_gender_cpd('LegalOrganization')
    
    builders['Sector'] = ImprovedBayesianCPTBuilder("../../data/randomized marginal tables/sector_gender_randomized.csv")
    cpds['Sector'] = builders['Sector'].build_feature_given_gender_cpd('Sector')
    
    gender_proportions = builders['AgeGroup'].df.groupby('Gender')['Proportion'].sum()
    cpd_gender = TabularCPD(
        variable='Gender',
        variable_card=2,
        values=[[gender_proportions['FEMALES']], [gender_proportions['MALES']]],
        state_names={'Gender': ['FEMALES', 'MALES']}
    )
    cpd_gender.normalize()
    cpd_income = builders['AgeGroup'].build_income_given_feature_gender_cpd('AgeGroup')
    
    model = DiscreteBayesianNetwork()
    model.add_edges_from([
        ('Gender', 'AgeGroup'),
        ('Gender', 'Industry'),
        ('Gender', 'CorpSize'),
        ('Gender', 'EmploymentType'),
        ('Gender', 'LegalOrganization'),
        ('Gender', 'Sector'),
        ('AgeGroup', 'IncomeLevel')
    ])
    
    model.add_cpds(cpd_gender)
    model.add_cpds(cpds['AgeGroup'], cpds['Industry'], cpds['CorpSize'])
    model.add_cpds(cpds['EmploymentType'], cpds['LegalOrganization'], cpds['Sector'])
    model.add_cpds(cpd_income)
    
    return model, builders

print("Building improved Bayesian network...")
improved_model, builders = build_improved_bayesian_network()

# Check model validity
print(f"Model validity check: {improved_model.check_model()}")

# Show network structure
print("\nNetwork structure:")
for edge in improved_model.edges():
    print(f"  {edge[0]} -> {edge[1]}")


In [ ]:
from pgmpy.sampling import BayesianModelSampling
sampling_inference = BayesianModelSampling(improved_model)

samples_improved = sampling_inference.forward_sample(size=100000)
income_dist_improved = samples_improved['IncomeLevel'].value_counts(normalize=True).sort_index()
print(income_dist_improved)

samples_improved.to_csv("../../data/synthetic/BN_improved_samples_100000.csv", index=False)
print("\nSaved: ../../data/synthetic/BN_improved_samples_100000.csv")

print("\n=== Income Distribution Comparison ===")
print("Original Model (Uniform Distribution Issue):")
if 'test_data' in locals():
    original_dist = test_data['IncomeLevel'].value_counts(normalize=True).sort_index()
    print(original_dist)
else:
    print("Original model data not available.")

print("\nImproved Model:")
print(income_dist_improved)


In [ ]:
print("=== Feature Distribution Analysis ===")
for feature in ['AgeGroup', 'Industry', 'CorpSize', 'EmploymentType', 'LegalOrganization', 'Sector']:
    print(f"\n{feature} Distribution:")
    feature_dist = samples_improved[feature].value_counts(normalize=True).head(5)
    print(feature_dist)

print(f"\nGender Distribution:")
gender_dist = samples_improved['Gender'].value_counts(normalize=True)
print(gender_dist)

print(f"\n=== Income and Age Relationship Analysis ===")
income_age_cross = pd.crosstab(samples_improved['AgeGroup'], samples_improved['IncomeLevel'], normalize='index')
print("Income level distribution across different age groups (row normalized):")
print(income_age_cross.round(3))

print(f"\n=== Income and Gender Relationship Analysis ===")
income_gender_cross = pd.crosstab(samples_improved['Gender'], samples_improved['IncomeLevel'], normalize='index')
print("Income level distribution across different genders (row normalized):")
print(income_gender_cross.round(3))

print(f"\n=== Model Consistency Check ===")
print(f"Total samples: {len(samples_improved)}")
print(f"Income levels: {samples_improved['IncomeLevel'].nunique()}")
print(f"Age groups: {samples_improved['AgeGroup'].nunique()}")
print(f"Industries: {samples_improved['Industry'].nunique()}")
print(f"Company sizes: {samples_improved['CorpSize'].nunique()}")
print(f"Employment types: {samples_improved['EmploymentType'].nunique()}")
print(f"Legal organizations: {samples_improved['LegalOrganization'].nunique()}")
print(f"Departments: {samples_improved['Sector'].nunique()}")
